Here, you develop the E+M Runge-Kuttas. (You want to be able to graph two functions at a time. The plotting functions can handle it, but the other functions aren't built for it specifically because your cases are simple enough that the answer for the B field is proportional to the E Field.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

In [2]:
nh_min = 64
refRatio = 2
CFL = np.sqrt(0.5)
ntC = 50
nt = int(2*ntC)
x_0 = 0.
x_1 = 0.5
Hans = False

deriv = 'U'
RK = 1
order = 1

L = 1
locs = [x_1]
epsilons = [1, 0.5] # [1, 1]
mus = [1, 0.5] # [1, 1]

func = WFT.Gauss # SquareWave
plotCont = True
enlarge = False

In [3]:
if (RK == 1):
    TimeIntegratorFunc = ST.ForwardEuler
else:
    if (RK == 2):
        TimeIntegratorFunc = ST.MidpointMeth
    else:
        TimeIntegratorFunc = ST.RK4

if (deriv == 'U'):
    # DiffMatFunc = OT.Upwind1D
    DiffFunc = TT.Upwind#ST.Upwind
else:
    # DiffMatFunc = OT.CenterDiff1D
    DiffFunc = TT.CenterDiff#ST.CenterDiff

In [4]:
def Block(matrica, var = 1):
    matricaList = [matrica for k in range(var)]
    matrice = block_diag(*matricaList)
    return matrice

In [5]:
omegaAMR = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh / refRatio))
omegaAMR.AddPatch(refRatio, finehalf)
degFreed = omegaAMR.degFreed
nh_max = omegaAMR.nh_max

omegaF = BT.Grid(nh_max)
omegaC = BT.Grid(nh_min)

In [6]:
physicsAMR = BT.PhysProps(omegaAMR, epsilons, mus, locs, L)
c = physicsAMR.cVec
cMatAMR = physicsAMR.cMat

physicsC = BT.PhysProps(omegaC, epsilons, mus, locs, L)
cC = physicsC.cVec
cMatC = physicsC.cMat

physicsF = BT.PhysProps(omegaF, epsilons, mus, locs, L)
cF = physicsF.cVec
cMatF = physicsF.cMat

cVecC = physicsC.cVec
c1 = cVecC[0]
c2 = cVecC[-1]

distance is
 [ 0.5        0.4921875  0.484375   0.4765625  0.46875    0.4609375  0.453125   0.4453125  0.4375     0.4296875  0.421875   0.4140625  0.40625    0.3984375  0.390625   0.3828125  0.375      0.3671875  0.359375   0.3515625  0.34375    0.3359375  0.328125   0.3203125  0.3125     0.3046875  0.296875   0.2890625  0.28125    0.2734375  0.265625   0.2578125  0.25       0.2421875  0.234375   0.2265625  0.21875    0.2109375  0.203125   0.1953125  0.1875     0.1796875  0.171875   0.1640625  0.15625    0.1484375  0.140625   0.1328125  0.125      0.1171875  0.109375   0.1015625  0.09375    0.0859375  0.078125   0.0703125  0.0625     0.0546875  0.046875   0.0390625  0.03125    0.0234375  0.015625   0.0078125  0.        -0.015625  -0.03125   -0.046875  -0.0625    -0.078125  -0.09375   -0.109375  -0.125     -0.140625  -0.15625   -0.171875  -0.1875    -0.203125  -0.21875   -0.234375  -0.25      -0.265625  -0.28125   -0.296875  -0.3125    -0.328125  -0.34375   -0.359375  -0.375     -0.3906

In [7]:
xC = omegaC.xNode
dxC = omegaC.dx[0]
width = 8 * dxC
center = 0.5 - (width / 2) - dxC
cellAve = True

if (plotCont):
    strPiece = 'Cont'
else:
    strPiece = ''

If  
$v_{j}^{s} = \frac{E_{j}^{s} - c B_{j}^{s}} {2 c}$  
and  
$v_{n + j}^{s} = \frac{E_{j}^{s} + c B_{j}^{s}}{2 c}$  
and  
$B_{j}^{0} = \frac{E_{j}^{0}}{c}$,  
then  
$c \Delta t \frac{\partial v_{j}^{0}}{\partial x} = 0$  
and  
$c \Delta t \frac{\partial v_{n + j}^{0}}{\partial x} = \frac{\sigma_{CFL}}{c} (E_{j}^{0} - E_{j - 1}^{0})$.  
  
If  
$v_{j}^{1} = v_{j}^{0} + c \Delta t \frac{\partial v_{j}^{0}}{\partial x}$  
and  
$v_{n + j}^{1} = v_{n + j}^{0} - c \Delta t \frac{\partial v_{n + j}^{0}}{\partial x}$,  
then  
$v_{j}^{1} = 0$  
and  
$v_{n + j}^{1} = \frac{1}{c} [(1 - \sigma_{CFL}) E_{j}^{0} + \sigma_{CFL} E_{j - 1}^{0}]$. 
  
If  
$E_{j}^{s} = c (v_{j}^{s} + v_{n + j}^{s})$  
and  
$B_{j}^{s} = v_{n + j}^{s} - v_{j}^{s}$,  
then  
$E_{j}^{1} = E_{j}^{0} - \sigma_{CFL} (E_{j}^{0} - E_{j - 1}^{0})$  
and  
$B_{j}^{1} = \frac{1}{c} [E_{j}^{0} - \sigma_{CFL} (E_{j}^{0} - E_{j - 1}^{0})] = B_{j}^{0} - \sigma_{CFL} (B_{j}^{0} - B_{j - 1}^{0})$.  


In [8]:
def Check(omega, physics, CFL, u0stack, v_i, v_f, u_f, op):
    
    degFreed = omega.degFreed
    nh_min = omega.nh_min
    c = physics.cVec[0]
    cMat = physics.cMat
    roundFact = 15
    
    I = np.eye(degFreed)
    dx, dt = ST.FindDxDt(omega, CFL, cMat)
    print('')
    print('')
    print('After 1 Forward Euler timestep on a uniform grid:')
    print('')
    print('The derivative of v should be')
    uShiftM = np.roll(u0stack, 1)
    uShiftP = np.roll(u0stack, -1)
    alpha_min = (1. - c) / (2 * c)
    alpha_plu = (1. + c) / (2 * c)
    derivShould1 = np.zeros(degFreed, float)
    derivShould2 = CFL * (uShiftM - u0stack) # The factor of c is already embedded within u0stack!!!
    derivShould = np.round(np.asarray(np.append(derivShould1[:degFreed], derivShould2[degFreed:])), roundFact)
    print(derivShould)
    print('In actuality, it is')
    derivres = np.round(dt * op @ v_i, roundFact)
    print(derivres)
    print('This is off by a factor of')
    r1 = np.round([derivShould[i] / derivres[i] for i in range(len(derivres))], 12)
    r1[derivShould == derivres] = 1
    print(r1)
    indices1 = np.where(r1 != 1)
    print('Indices:', indices1[0])
    print('Factors:', r1[indices1[0]])
    print('')
    print('v itself should be')
    shouldbe1 = np.zeros(degFreed, float) # alpha_min * (((1. - CFL) * u0stack) + (CFL * uShiftP))
    shouldbe2 = ((1. - CFL) * u0stack) + (CFL * uShiftM) # The factor of c is already embedded within u0stack!!!
    shouldbe = np.round(np.asarray(np.append(shouldbe1[:degFreed], shouldbe2[degFreed:])), roundFact)
    print(shouldbe)
    print('In actuality, it is')
    v_f1 = np.round(v_f, roundFact)
    print(v_f1)
    print('This is off by a factor of')
    r = np.round([shouldbe[i] / v_f1[i] for i in range(len(v_f))], roundFact)
    r[shouldbe == v_f1] = 1
    print(r)
    indices = np.where(r != 1)
    print('Indices:', indices[0])
    print('Factors:', r[indices[0]])
    print('')
    print('u should be')
    u1 = ((1. - CFL) * u0stack) + (CFL * uShiftM) # The factor of c is already embedded within u0stack!!!

    u = np.round(u1, roundFact)
    print(u)
    print('In actuality, it is')
    u_f1 = np.round(u_f, roundFact)
    print(u_f1)
    print('This is off by a factor of')
    r2 = np.round([u[i] / u_f1[i] for i in range(len(u_f))], roundFact)
    r2[u == u_f1] = 1
    print(r2)
    indices2 = np.where(r2 != 1)
    print('Indices:', indices2[0])
    print('Factors:', r2[indices2[0]])
    return

In [9]:
# Creates anti-block-diagonal matrix 
def ConvertOp(omega, physics):
    degFreed = omega.degFreed
    cMat = physics.cMat
    cVec = physics.cVec
    c1 = cVec[0]
    c2 = cVec[-1]
    
    bacCoefs = np.zeros(degFreed, float)
    forCoefs = np.ones(degFreed, float)
    bacCoefsInv = np.zeros(degFreed, float)
    forCoefsInv = np.ones(degFreed, float)
    
    cVecs = np.asarray(np.append(cVec, -cVec))
    
    I = np.eye(degFreed)
    
    # THE CHECK IS NOT SET UP TO HANDLE NON-ONE VALUES FOR normConst!
    normConst = 1. # 0.5 #  / np.sqrt(c**2 + 1)
    op = normConst * np.asarray(np.block([[cMat, cMat], [-I, I]]))
    diagMat = np.diag(cVecs)
    
    zeros = np.zeros((degFreed, degFreed), float)
    
    mainOp = np.asarray(np.block([[zeros, -cMat@cMat], [-I, zeros]]))
    
    RCoef = (c2 - c1) / (c1 + c2)
    TCoef = (2. * c1) / (c1 + c2)
    
    bacCoefs[cVec == c1] = RCoef
    forCoefs[cVec == c2] = TCoef
    
    bacCoefsInv[cVec == c1] = 1. / RCoef
    forCoefsInv[cVec == c2] = 1. / TCoef
    
    Coefs = np.asarray(np.append(bacCoefs, forCoefs))
    CoefsInv = np.asarray(np.append(bacCoefsInv, forCoefsInv))
    
    # THE CHECK IS NOT SET UP TO HANDLE NON-IDENTITY MATRICES FOR CoefOp!
    CoefOp = np.eye(2 * degFreed) # np.diag(Coefs) #
    CoefOpInv = np.eye(2 * degFreed) # np.diag(CoefsInv) #

    return op, diagMat

In [10]:
def RKPieces(omega, physics, CFL, RK, deriv, order, PDEOrder):
    
    errorLoc = 'ERROR:\nSolverTools:\nRKPieces:\n'
    errorMess = ''
    matInd = -1#physics.matInd
    
    if (deriv == 'F'):
        left = False
        derivOp1 = OT.ExactSpatDerivOp(omega)
        derivOp2 = derivOp1
    else:
        left = True
        if (deriv == 'U' or deriv == 'UD'):
            derivOp1 = OT.SpaceDeriv(omega, order, 'D', matInd0 = matInd)
            derivOp2 = OT.SpaceDeriv(omega, order, deriv, matInd0 = matInd)
        else:
            if (deriv == 'D' or deriv == 'DD'):
                derivOp1 = OT.SpaceDeriv(omega, order, 'U', matInd0 = matInd)
                derivOp2 = OT.SpaceDeriv(omega, order, deriv, matInd0 = matInd)
            else:
                # Not sure if the negative signs here are necessary.
                derivOp1 = -OT.SpaceDeriv(omega, order, deriv, matInd0 = matInd)
                derivOp2 = -derivOp1
    
    if (RK == 1):
        Scheme = ForwardEuler
    else:
        if (RK == 2):
            Scheme = ST.MidpointMeth
        else:
            if (RK == 4):
                Scheme = ST.RK4
            else:
                errorMess = str(RK) + ' is not a valid RK entry!'
    
    cMat1 = physics.cMat
    dx, dt = ST.FindDxDt(omega, CFL, cMat1)
    waves1 = WT.MakeWaves(omega)
    nullspace = OT.FindNullspace(omega, waves1)
    
    
    if (PDEOrder == 1):
        derivOp == derivOp2
        waves = waves1 @ nullspace
        cMat = cMat1
    else:
        if (PDEOrder == 2):
            derivOp = OT.Block([derivOp1, derivOp2])
#             derivOp = OT.Block([-derivOp2.transpose(), derivOp2]) # ExpandMat(derivOp)
            waves = OT.Block(waves1 @ nullspace, var = 2)
            cMat = OT.Block(cMat1, var = 2)
        else:
            errorMess = 'Invalid PDE order!'
    
    if (errorMess != ''):
        sys.exit(errorLoc + errorMess)
    
    return dt, cMat, waves, derivOp, Scheme, left


In [11]:
def WaveEqRK(omega, physics, u0, nt, CFL, RK, deriv, order):
    
    errorLoc = 'ERROR:\nSolverTools:\nWaveEqRK:\n'
    errorMess = ''
    
    dt, cMat, waves, derivOp, Scheme, left = RKPieces(omega, physics, CFL, RK, deriv, order, 2)
    cMat1 = physics.cMat
    cMatInv1 = LA.inv(cMat1)
    
    # Unpack requisite attributes.
    degFreed = omega.degFreed
    c = cMat[0, 0]
    l = omega.levels
    
    # Stack the initial condition.
    u0stack = np.asarray(np.append(u0, cMatInv1 @ u0)) # I'm not sure how to relate the initial condition for the E and B fields.
    
    # Create change of basis matrix.
    convertOp, diagMat = ConvertOp(omega, physics)
    convertOpinv = LA.inv(convertOp)
    print('inverse of op is')
    print(convertOpinv)
    
    # Change basis of initial condtion.
    v = np.asarray(convertOpinv @ u0stack)
    v_i = v.copy()  
    
    # Create identity matrix to appease the current structure of my RK function.
    I = np.eye(2 * degFreed)
    
    # Create operator to pass into RK function.
    op = diagMat @ derivOp
    
    print('')
    print('op is')
    print(op)
    print('')
    
    t = 0
    for n in range(nt):
        v, t = Scheme(v, t, dt, op, waves, left)

#     for i in range(nt):
#         v = ST.RungeKutta(omega, physics, I, v, 1, CFL, RK, op = op)
    
    # Convert v back to space-space.
    u = convertOp @ v

    # Convert u to k space.
    fCoefs = LA.inv(waves) @ u
    
    # Run check.
    if ((nt == 1) and (RK == 1) and (l == 0)):
        Check(omega, physics, CFL, u0stack, v_i, v, u, op)
    
    return fCoefs, v

In [12]:
def MaxwellRHS(omega, phyiscs, u0):
    
    convertOp, diagMat = ConvertOp(omega, physics)
    convertOpInv = LA.inv(convertOp)
    
    phi = convertOpInv @ u0
    
    
    return

In [13]:
def ForwardEuler(omega, physics, u0, t, dt, op, waves, left = True, charOp = []): #(omega, waves, u0, nt, const, CFL, func, order = 0):
    if (left):
        func = LeftMult
    else:
        func = FDeriv

    u = u0 + (dt * func(t, u0, op, waves, charOp))
    t = t + dt
#         if (func == TimePoly):
#             if (n == nt - 1):
#                 val = Operate(t, u, op) # func(omega, t, u, const, order + 1, deriv = False)
#                 if (order < 2):
#                     midstring = ' should be equal to '
#                 else:
#                     midstring = ' does not necessarily need to equal '
#                 print(str(u[0]) + midstring + str(val) + '.')
    return u, t

In [14]:
def MaxwellRK(omega, physics, u0, nt, CFL, RK, deriv, order):
    
    dt, cMat, waves, derivOp, Scheme, left = RKPieces(omega, physics, CFL, RK, deriv, order, 2)
    cMat1 = physics.cMat
    cMatInv1 = LA.inv(cMat1)
    
    # Unpack requisite attributes.
    degFreed = omega.degFreed
    c = cMat[0, 0]
    l = omega.levels
    
    # Stack the initial condition.
    u0stack = np.asarray(np.append(u0, cMatInv1 @ u0)) # I'm not sure how to relate the initial condition for the E and B fields.
    
    
    
    for n in range(nt):
        v, t = Scheme(omega, physics, v, t, dt, op, waves, left)
    
    
    
    return

In [15]:
# # THIS ONE IS MEANT TO BE USED WITH THE OTHER FORWARD EULER!!!
# def WaveEqRK(omega, physics, u0, nt, CFL, RK, deriv, order):
    
#     errorLoc = 'ERROR:\nSolverTools:\nWaveEqRK:\n'
#     errorMess = ''
    
#     dt, cMat, waves, derivOp, Scheme, left = RKPieces(omega, physics, CFL, RK, deriv, order, 2)
#     cMat1 = physics.cMat
#     cMatInv1 = LA.inv(cMat1)
    
#     # Unpack requisite attributes.
#     degFreed = omega.degFreed
#     c = cMat[0, 0]
#     l = omega.levels
    
#     # Stack the initial condition.
#     u = np.asarray(np.append(u0, cMatInv1 @ u0)) # I'm not sure how to relate the initial condition for the E and B fields.
    
#     # Create change of basis matrix.
#     convertOp, diagMat = ConvertOp(omega, physics)
#     convertOpinv = LA.inv(convertOp)
#     print('inverse of op is')
#     print(convertOpinv)
    
#     # Change basis of initial condtion.
# #     v = np.asarray(convertOpinv @ u0stack)
# #     v_i = v.copy()  
    
#     # Create identity matrix to appease the current structure of my RK function.
#     I = np.eye(2 * degFreed)
    
#     # Create operator to pass into RK function.
#     op = diagMat @ derivOp
    
#     t = 0
#     for n in range(nt):
#         u, t = Scheme(u, t, dt, op, waves, left, charOp = convertOp)

#     # Convert u to k space.
#     fCoefs = LA.inv(waves) @ u
    
#     # Run check.
#     if ((nt == 1) and (RK == 1) and (l == 0)):
#         Check(omega, physics, CFL, u0stack, v_i, v, u, op)
    
#     return fCoefs, u

In [16]:
# c = ConvertParams(permitivity, permeability, L)
t, nt = ST.CalcTime(omegaAMR, CFL, cMatAMR, nt = nt)
print(t)
mu, sigma = WFT.GaussParams(x_0, x_1)
args = [mu, sigma] # [width, center] # 
wavesAMR = WT.MakeWaves(omegaAMR)
nullspace = OT.FindNullspace(omegaAMR, wavesAMR, Hans = Hans)
nullspaceBlock = OT.Block(nullspace, var = 2)
restrictOp = GTT.CoarsenOp(omegaAMR)
print('sig:', sigma, 'mu:', mu)

wavesF = WT.MakeWaves(omegaF)
wavesC = WT.MakeWaves(omegaC)

derivMat = TT.ExactSpatOp(omegaAMR)
print(np.shape(cMatF))
print(np.shape(derivMat))
spatOp = -cMatF @ derivMat
timePropOp = LA2.expm(t * spatOp)

derivMatC = TT.ExactSpatOp(omegaC)
spatOpC = -cMatC @ derivMatC
timePropOpC = LA2.expm(t * spatOpC)


0.27621358640099514
sig: 0.25 mu: 0.03113530673812129
You are using ExactSpatOp in TestTools module!
(128, 128)
(128, 128)
You are using ExactSpatOp in TestTools module!


In [17]:
# wavesAMR = OT.Block(wavesAMR, var = 2)
# nullspace = OT.Block(nullspace, var = 2)
# restrictOp = OT.Block(restrictOp, var = 2)
# wavesC = OT.Block(wavesC, var = 2)
# wavesF = OT.Block(wavesF, var = 2)

opC = OT.SpaceDeriv(omegaC, order, deriv)#-cMatC @ OT.SpaceDeriv(omegaC, order, diff)
opF = OT.SpaceDeriv(omegaF, order, deriv)#-cMatF @ OT.SpaceDeriv(omegaF, order, diff)
opAMR = OT.SpaceDeriv(omegaAMR, order, deriv)#-cMatAMR @ OT.SpaceDeriv(omegaAMR, order, diff)

In [18]:
# For Gaussian:
waveInitC = WFT.InitCond(omegaC, physicsC, func, args)
# waveInitC = np.append(waveInitC, waveInitC)
waveInitF = WFT.InitCond(omegaF, physicsF, func, args)
# waveInitF = np.append(waveInitF, waveInitF)

FCoefsC = FFTT.FourierCoefs(wavesC, waveInitC)
FCoefsF = FFTT.FourierCoefs(wavesF, waveInitF)
FCoefsC1 = np.asarray(np.append(FCoefsC, FCoefsC / c1))
FCoefsF1 = np.asarray(np.append(FCoefsF, FCoefsF / c1))

# Find initial waveform on coarse, fine, and AMR grids.
waveInitC = wavesC @ FCoefsC
waveInitF = wavesF @ FCoefsF
waveInitAMR = restrictOp @ waveInitF

# Find Fourier coefficients for initial condition on AMR grid.
FCoefsAMR = FFTT.FourierCoefs(wavesAMR @ nullspace, waveInitAMR)
FCoefsAMR1 = np.asarray(np.append(FCoefsAMR, FCoefsAMR / c1))

EwavePropC = WFT.WaveEq(omegaC, physicsC, func, args, t)#FFTT.FourierCoefs(wavesC, wavePropC) # timePropOpC @ FCoefsC
EwavePropF = WFT.WaveEq(omegaF, physicsF, func, args, t)
EwavePropAMR = WFT.WaveEq(omegaAMR, physicsAMR, func, args, t)#FFTT.FourierCoefs(wavesC, wavePropC) # timePropOpC @ FCoefsC

BwavePropC = WFT.WaveEq(omegaC, physicsC, func, args, t, field = 'B') # / c1#FFTT.FourierCoefs(wavesC, wavePropC) # timePropOpC @ FCoefsC
BwavePropF = WFT.WaveEq(omegaF, physicsF, func, args, t, field = 'B') # / c1
BwavePropAMR = WFT.WaveEq(omegaAMR, physicsAMR, func, args, t, field = 'B') # / c1

# Find propagated coeficients on coarse and fine grid.
EFCoefsPropC = FFTT.FourierCoefs(wavesC, EwavePropC)
EFCoefsPropF = FFTT.FourierCoefs(wavesF, EwavePropF)
EFCoefsPropAMR = FFTT.FourierCoefs(wavesAMR @ nullspace, EwavePropAMR)

BFCoefsPropC = FFTT.FourierCoefs(wavesC, BwavePropC)
BFCoefsPropF = FFTT.FourierCoefs(wavesF, BwavePropF)
BFCoefsPropAMR = FFTT.FourierCoefs(wavesAMR @ nullspace, BwavePropAMR)

FCoefsPropC1 = np.asarray(np.append(EFCoefsPropC, BFCoefsPropC))
FCoefsPropF1 = np.asarray(np.append(EFCoefsPropF, BFCoefsPropF))
FCoefsPropAMR1 = np.asarray(np.append(EFCoefsPropAMR, BFCoefsPropAMR))

# TEST FOR EXACT GAUSSIAN AT NEW LOCATION.

# solverCoefsC = ST.RungeKutta(omegaC, physicsC, wavesC, waveInitC, nt, CFL, RK, op = opC) # TimeIntegratorFunc(omegaC, wavesC, waveInitC, nt, cMatC, CFL, DiffFunc)
# solverCoefsF = ST.RungeKutta(omegaF, physicsF, wavesF, waveInitF, nt, CFL, RK, op = opF) # TimeIntegratorFunc(omegaF, wavesF, waveInitF, nt, cMatF, CFL, DiffFunc)
# solverCoefsAMR = ST.RungeKutta(omegaAMR, physicsAMR, wavesAMR @ nullspace, waveInitAMR, nt, CFL, RK, op = opAMR) # TimeIntegratorFunc(omegaAMR, wavesAMR @ nullspace, waveInitAMR, nt, cMat, CFL, DiffFunc, order = order)

# solverCoefsC = WaveEqRK(omegaC, physicsC, wavesC, waveInitC, nt, CFL, RK, derivOp = opC) # TimeIntegratorFunc(omegaC, wavesC, waveInitC, nt, cMatC, CFL, DiffFunc)
# solverCoefsF = WaveEqRK(omegaF, physicsF, wavesF, waveInitF, nt, CFL, RK, derivOp = opF) # TimeIntegratorFunc(omegaF, wavesF, waveInitF, nt, cMatF, CFL, DiffFunc)
# solverCoefsAMR = WaveEqRK(omegaAMR, physicsAMR, wavesAMR @ nullspace, waveInitAMR, nt, CFL, RK, derivOp = opAMR) # TimeIntegratorFunc(omegaAMR, wavesAMR @ nullspace, waveInitAMR, nt, cMat, CFL, DiffFunc, order = order)

solverCoefsC, vC = WaveEqRK(omegaC, physicsC, waveInitC, ntC, CFL, RK, deriv, order) # TimeIntegratorFunc(omegaC, wavesC, waveInitC, nt, cMatC, CFL, DiffFunc)
solverCoefsF, vF = WaveEqRK(omegaF, physicsF, waveInitF, nt, CFL, RK, deriv, order) # TimeIntegratorFunc(omegaF, wavesF, waveInitF, nt, cMatF, CFL, DiffFunc)
solverCoefsAMR, vAMR = WaveEqRK(omegaAMR, physicsAMR, waveInitAMR, nt, CFL, RK, deriv, order) # TimeIntegratorFunc(omegaAMR, wavesAMR @ nullspace, waveInitAMR, nt, cMat, CFL, DiffFunc, order = order)




allCoefsC = PT.Load(FCoefsC1, FCoefsPropC1, solverCoefsC)#, FCoefsPropCTh)
allCoefsF = PT.Load(FCoefsF1, FCoefsPropF1, solverCoefsF)#, FCoefsPropFTh)
allCoefsAMR = nullspaceBlock @ PT.Load(FCoefsAMR1, FCoefsPropAMR1, solverCoefsAMR)#, FCoefsPropAMRTh)




labels = ['Initial Wave', 'Exact Propagated Wave', 'RK Propagated Wave']#, 'Rotation Matrix Propagated Wave']


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 128 is different from 64)

In [ ]:
actual = np.round(FCoefsPropC1, 12)[nh_min:]
solver = np.round(solverCoefsC, 12)[nh_min:]

print(actual)
print(solver)
print(actual.transpose() @ actual)
print(solver.transpose() @ solver)
print('')

In [ ]:
PT.PlotMixedWave(omegaC, physicsC, waves = wavesC, FCoefs = allCoefsC, rescale = 5, yGrid = True, labels = labels, plotCont = plotCont, enlarge = enlarge, saveName = 'RKWaveEqAttempt2')#, saveName = 'Coarse', dpi = 300)
PT.PlotMixedWave(omegaF, physicsF, waves = wavesF, FCoefs = allCoefsF, rescale = 5, yGrid = True, labels = labels, enlarge = enlarge, plotCont = plotCont)#, title = 'Fine-Grid Mode Propagation', saveName = 'Fine', dpi = 300)
PT.PlotMixedWave(omegaAMR, physicsAMR, waves = wavesAMR, FCoefs = allCoefsAMR, rescale = 5, yGrid = True, labels = labels, enlarge = enlarge, plotCont = plotCont)#, title = 'AMR-Grid Mode Propagation', saveName = 'AMRHansMethod', dpi = 300)


In [ ]:
xC = omegaC.xCell
print('THIS IS INCORRECT!!!')
plt.figure()
plt.plot(xC, vC[:nh_min], label = 'Backward Eigenvector')
plt.plot(xC, vC[nh_min:], label = 'Forward Eigenvector')
plt.legend()
plt.title('Eigenvectors')
plt.savefig('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/Figures/Eigenvectors.png', bbox_inches = 'tight', transparent = False)
plt.show()

In [ ]:
# Notes for Sasha:
#
# Make residuals test of Gaussian (first unpropagated, then propagated) to assess its validity.


In [ ]:
waveMat = OT.Block(wavesC, var = 2)
solvRes = waveMat @ solverCoefsC
print(solvRes[:nh_min])
print(solvRes[nh_min:])
print('')